<a href="https://colab.research.google.com/github/farhad06/Cricket_comments/blob/main/Cricket_comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import nltk
import numpy as np
import sklearn
import string
nltk.download('punkt')
import re
from nltk.corpus import stopwords
# nltk.download('punkt')
# rom nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics,pipeline,feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.linear_model import LogisticRegression # for Logistic regression algo
from sklearn import svm ##for SVM algo
from sklearn.neighbors import KNeighborsClassifier #for KNN ALGO
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer #for stremming word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Cricket_comments/comments.xlsx")
words=pd.read_excel('/content/drive/MyDrive/Cricket_comments/BN_stop_words.xlsx')
# del df['Unnamed: 0']
df.head()

,Comment,opinion
0,আমি এটার সঙ্গে যাইনি এখানে পারি মাধ্যমে করা হব...,2
1,যাত্রার অবিশ্বাস্য সূচনা মাত্র,0
2,দ্বিতীয় টেস্টে নিউজিল্যান্ডকে ৩৭২ রানে পরাজিত ...,2
3,একেই বলে স্বপ্নের অভিষেক!,2
4,দেশের মাটিতে শেষ পাঁচটি টেস্ট ইনিংসে চারটি হাফ...,2


In [ ]:
df.head()

,Comment,opinion
0,আমি এটার সঙ্গে যাইনি এখানে পারি মাধ্যমে করা হব...,2
1,যাত্রার অবিশ্বাস্য সূচনা মাত্র,0
2,দ্বিতীয় টেস্টে নিউজিল্যান্ডকে ৩৭২ রানে পরাজিত ...,2
3,একেই বলে স্বপ্নের অভিষেক!,2
4,দেশের মাটিতে শেষ পাঁচটি টেস্ট ইনিংসে চারটি হাফ...,2


In [ ]:
df.tail()

,Comment,opinion
1993,জাতীয় দলে আসতে হলে আশরাফুল কে যোগ্যতা দিয়েই আস...,2
1994,"""শুভাগত হোমের মতো অফ ফর্মের প্লেয়ার এতো দ্রুত ...",1
1995,নতুন কোন খেলোয়াড়ই দেখছি না । নতুন খেলোয়াড়রা না...,0
1996,"দুর্দান্ত রেকর্ডেরর জন্য কোহলীকে অভিনন্দন, সাথ...",2
1997,তাহলে আবার প্রমাণ হল যে কোহলি সেঞ্চুরি করলেই ভ...,1


In [ ]:
df.shape

(1998, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  1998 non-null   object
 1   opinion  1998 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.3+ KB


In [ ]:
df.describe()

,opinion
count,1998.000000
mean,1.138639
std,0.812582
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,2.000000


In [ ]:
words.head()

,Words
0,অতএব
1,অথচ
2,অথবা
3,অনুযায়ী
4,অনেক


In [ ]:
df.isna().sum()

Comment    0
opinion    0
dtype: int64

In [ ]:
stop_words=set(words['Words'])
print(stop_words)

{'পরে', 'তবু', 'চার', 'এটি', ',', 'কাউকে', 'করেছেন', 'প্রায়', 'লক্ষ', 'কিংবা', 'এ', 'আপনার', 'বা', 'ভাবেই', 'জ্নজন', 'আমাদের', 'তেমন', 'নেওয়া', 'করিয়ে', 'তাঁদের', 'তিনি', 'বক্তব্য ', 'জানানো', 'অবধি', 'দ্বারা', 'করিতে', 'রয়েছে', 'কে', 'দেন', 'মধ্যে', 'করেছিলেন', 'পারেন', 'হওয়া', '|', 'নয়', 'যেন', 'বলা', 'বসে', 'জানতে', 'গেছে', 'হবেন', 'আপনি', 'তাঁকে', 'এঁরা', 'একটি', 'নাগাদ', 'শুরু ', 'দিন', 'কাছে', 'চেয়ে', 'শুধু ', 'বলল', 'করতে', 'চেয়ে', 'অবশ্য', 'উপর', 'মধ্যেও', 'একবার', 'বলে', 'সেই', 'তাহাতে', 'হতেই', 'বেশি', 'নাকি', 'আজ', 'দুটি', 'করি', 'প্রথম ', 'বিষয়টি', 'বললেন', 'একই', 'থাকা', 'করবেন', 'সঙ্গে', 'পরেই', 'গিয়ে', 'দেয়', '…..', 'কাজে', 'দিয়েছে', 'হয়নি', 'অনুযায়ী', 'ফিরে', 'থাকায়', 'হয়ে', 'গেলে', 'জানিয়ে', 'ক্ষেত্রে', 'ধরে', 'যাঁরা', 'যায়', 'কারণ ', 'তত', 'হতে', 'যারা', 'হত', 'করেছে', 'হওয়ার', 'যাতে', 'করবে', 'যাওয়া', 'হওয়ায়', 'পারে', 'যা', 'বাদে', 'চায়', 'খুব', 'রাখা', 'করে', 'দু', 'এখানে', 'অনেকে', 'ই ', 'ঐ', 'কয়েকটি', 'জানায়', 'হলো', 'নিজের', 'কবে', 'রেখে', 'হন', 

In [ ]:
df['opinion'].value_counts()

2    817
1    641
0    540
Name: opinion, dtype: int64

In [ ]:
##remove punctuation
def remove_punctuation(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    punctuationfree="".join([i for i in text if i not in punctuations])
    return punctuationfree

In [ ]:
##remove stop words
def stop_word(tokens):
    stop_word_free=" ".join([w for w in tokens if w not in stop_words])
        #print(stop_word_free)  
    return stop_word_free

In [ ]:
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

In [ ]:
clean_data=df['Comment'].apply(lambda x:remove_punctuation(x))
token_data = clean_data.apply(lambda x: tokenization(x))
sw_free_data=token_data.apply(lambda x:stop_word(x))

In [ ]:
sw_free_data.values

array(['এটার যাইনি', 'যাত্রার অবিশ্বাস্য সূচনা',
       'দ্বিতীয় টেস্টে নিউজিল্যান্ডকে ৩৭২ রানে পরাজিত ১০ ব্যবধানে টেস্ট সিরিজ জয়লাভ করলো টিম ইন্ডিয়া।।',
       ...,
       'খেলোয়াড়ই দেখছি । খেলোয়াড়রা উঠে আসলে বিপিএল স্বার্থ বাণিজ্যিকই । নতুনদের দেশের বিদেশের সেরা সেরা খেলোয়াড়দের সাথে পারফর্ম ভাল সুযোগ প্লাটফর্ম ।',
       'দুর্দান্ত রেকর্ডেরর কোহলীকে অভিনন্দন সাথে বৎসর ভারতের মাটিতে ওয়ান্ডে জয়ের অভিনন্দন ওয়েস্ট ইন্ডিজকে',
       'প্রমাণ কোহলি সেঞ্চুরি করলেই ভারত হারে। আসলে খেলে। দেশের খেললে জিতবে যাই পিচও ভারতকে বাঁচাতে পারল না। হার একেবারে ৪৩ রানের বিশাল পরাজয়। পৃথিবীর অন্যতম দুর্বল দলের দেশের মাঠে। হাহাহাহাহা। পুরো ৫০ ওভারও ব্যাট'],
      dtype=object)

In [ ]:
corpus=sw_free_data.values
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
y=df['opinion'].values

In [ ]:
len(y)

1998

Naive Bayes 

In [ ]:
classifier_model = naive_bayes.MultinomialNB()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [ ]:
y_train.shape,y_test.shape

((1498,), (500,))

In [ ]:
classifier_model.fit(X_train,y_train)
pred=classifier_model.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, pred)

0.456

**Decision Tree**

---



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy of Decision Tree:",metrics.accuracy_score(y_test, y_pred))

Accuracy of Decision Tree: 0.36833333333333335


**Logistic Regression**





In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
print("Accuracy of  Logistic Regression :",metrics.accuracy_score(y_test, y_pred))

Accuracy of  Logistic Regression : 0.452


**SVM**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=109) 

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy of  Support vector machine :",metrics.accuracy_score(y_test, y_pred))

Accuracy of  Support vector machine : 0.4116666666666667


**KNN**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
print("Accuracy of  KNN :",metrics.accuracy_score(y_test, y_pred))

Accuracy of  KNN : 0.3575


**Split the data**

In [ ]:
### split the data
from sklearn.model_selection import train_test_split
review = df['Comment'].values
label = df['opinion'].values
review_train, review_test, label_train, label_test = train_test_split(
    review, label, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train)
Xlr_train = review_vectorizer.transform(review_train)
Xlr_test  = review_vectorizer.transform(review_test)
Xlr_train

<1498x1502 sparse matrix of type '<class 'numpy.int64'>'
	with 14353 stored elements in Compressed Sparse Row format>

**CNN**

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  
print(review_train[1])
print(Xcnn_train[1]) 

মাশরাফি হলো বাংলাদেশ ক্রিকেটের ম্যাজিক স্টিক।
[299, 185, 15, 128, 2785, 2786]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :])

[  92  184    2 2783   10  134 2784 1721   21    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [ ]:
from keras.models import Sequential
from keras import layers

In [ ]:
##making model using layers
embedding_dim = 200
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 200)          1465200   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           128128    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,594,629
Trainable params: 1,594,629
Non-trainable params: 0
______________________________________________

In [ ]:
## fit the model and check accurency
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=10,
                     verbose=False,
                     validation_data=(Xcnn_test, label_test),
                     batch_size=50)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.3224
Testing Accuracy:  0.3160
